In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Define image size and baych size
IMG_SIZE = 224
BATCH_SIZE = 32


In [10]:
#Define data generators for train, validation and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/test',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 2407 images belonging to 2 classes.
Found 601 images belonging to 2 classes.
Found 60 images belonging to 1 classes.


In [11]:
# Define the model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [12]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
history = model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
76/76 [==============================] - 731s 10s/step - loss: 0.5167 - accuracy: 0.7823 - val_loss: 0.2691 - val_accuracy: 0.8935
Epoch 2/5
76/76 [==============================] - 277s 4s/step - loss: 0.2053 - accuracy: 0.9206 - val_loss: 0.1207 - val_accuracy: 0.9584
Epoch 3/5
76/76 [==============================] - 294s 4s/step - loss: 0.0910 - accuracy: 0.9705 - val_loss: 0.0351 - val_accuracy: 0.9900
Epoch 4/5
76/76 [==============================] - 276s 4s/step - loss: 0.0600 - accuracy: 0.9830 - val_loss: 0.0865 - val_accuracy: 0.9834
Epoch 5/5
76/76 [==============================] - 275s 4s/step - loss: 0.0262 - accuracy: 0.9938 - val_loss: 0.0124 - val_accuracy: 0.9967


In [16]:
model.save("Model.h5","label.txt")

In [19]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
from tensorflow.keras.preprocessing import image
import numpy as np
#load the model
model = load_model('/content/Model.h5')
#Load andd preprocessor
test_image_path='/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/train/test/pred/pred14.jpg'
img=image.load_img(test_image_path,target_size=(224,224))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array,axis=0)#Add batch dimension
img_array/=255. #Normalize thepixel values

#Make prediction
prediction=model.predict(img_array)

#print the prediction
if prediction < 0.5:
  print("Prediction:No tumor (Probability:",prediction[0][0],")")
else:
  print("Prediction:Tumor present (Probability:",prediction[0][0],")")

1/1 [==============================] - 0s 161ms/step
Prediction:Tumor present (Probability: 0.99999666 )
